# 🤖 Basic Agents with Microsoft Agent Framework (Python)

## 📋 What You'll Learn

This notebook shows you how to create AI agents using the Microsoft Agent Framework. You'll see three different ways to create agents and how to have conversations with them.

**What's Inside:**
- 🏗️ **Three Agent Creation Methods**: GitHub Models, Azure OpenAI, and Azure AI Foundry
- 🛠️ **Tool Integration**: Adding custom functions agents can call
- 💬 **Conversation Management**: Single-turn and multi-turn conversations
- 🔄 **Streaming Responses**: Real-time response display

## ⚙️ Setup Requirements

### 1. **Docker & Dev Container**
- Install Docker on your machine
- Open this project in the dev container (VS Code will prompt you)

### 2. **Workspace Configuration**
- Open the workspace: `File > Open Workspace from File` → select `workspace.code-workspace`
- This sets up the correct Python environment for each project folder

### 3. **Environment Variables**
1. Copy `.env.example` to `.env` in the `agent-framework-samples` folder:
   ```bash
   cp .env.example .env
   ```
2. Fill in your credentials:
   ```env
   # For GitHub Models (Option 1)
   GITHUB_TOKEN=your_github_token
   GITHUB_ENDPOINT=https://models.inference.ai.azure.com
   GITHUB_MODEL_ID=gpt-4o-mini
   
   # For Azure OpenAI (Option 2)
   AZURE_OPENAI_ENDPOINT=your_endpoint
   AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=your_deployment
   
   # For Azure AI Foundry (Option 3)
   AZURE_AI_PROJECT_ENDPOINT=your_project_endpoint
   AZURE_AI_MODEL_DEPLOYMENT_NAME=your_model
   
   # For Local Ollama (Option 4)
   OLLAMA_ENDPOINT=http://localhost:11434/v1
   OLLAMA_DEPLOYMENT_NAME=llama3
   ```

## 🚀 What You'll Build

Below you'll find code samples demonstrating:

1. **Tool Definition**: A simple function that picks random travel destinations
2. **Agent Configuration**: Instructions that define agent behavior
3. **Three Creation Patterns**:
   - **Option 1**: GitHub Models (simplest, uses OpenAI-compatible API)
   - **Option 2**: Azure OpenAI (local agent, no persistence)
   - **Option 3**: Azure AI Foundry (persistent agent in the cloud)
4. **Conversation Examples**: Single-turn and multi-turn conversations with streaming

Let's start building! 🌟

In [28]:

import os                     
from random import randint    

from dotenv import load_dotenv  
from agent_framework import ChatAgent
from azure.identity import AzureCliCredential, DefaultAzureCredential
from agent_framework.azure import AzureOpenAIChatClient
from agent_framework.openai import OpenAIChatClient

from azure.ai.projects.aio import AIProjectClient
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient

load_dotenv()

True

In [29]:
# 🛠️ Tool Function Design Pattern
def get_random_destination() -> str:
    """Get a random vacation destination using Repository Pattern.
    
    Returns:
        str: A randomly selected destination following consistent format
    """
    destinations = [
        "Barcelona, Spain",      
        "Paris, France",         
        "Berlin, Germany",       
        "Tokyo, Japan",          
        "Sydney, Australia",     
        "New York, USA",         
        "Cairo, Egypt",          
        "Cape Town, South Africa", 
        "Rio de Janeiro, Brazil",  
        "Bali, Indonesia"          
    ]
    
    return destinations[randint(0, len(destinations) - 1)]

In [30]:
AGENT_NAME ="TravelAgent"

AGENT_INSTRUCTIONS = """You are a helpful AI Agent that can help plan vacations for customers.

Important: When users specify a destination, always plan for that location. Only suggest random destinations when the user hasn't specified a preference.

When the conversation begins, introduce yourself with this message:
"Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. Here are some things you can ask me:
1. Plan a day trip to a specific location
2. Suggest a random vacation destination
3. Find destinations with specific features (beaches, mountains, historical sites, etc.)
4. Plan an alternative trip if you don't like my first suggestion

What kind of trip would you like me to help you plan today?"

Always prioritize user preferences. If they mention a specific destination like "Bali" or "Paris," focus your planning on that location rather than suggesting alternatives.
"""

In [21]:
# Option 1: Using GitHub Models via OpenAIChatClient

chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"),
    model_id=os.environ.get("GITHUB_MODEL_ID"))

chat_agent = ChatAgent(
        name = AGENT_NAME,
        chat_client=chat_client,
        instructions=AGENT_INSTRUCTIONS,
        tools=[get_random_destination]
)

In [22]:
#Option 2: Using Azure OpenAi client to generate a local agent

chat_agent = AzureOpenAIChatClient(credential=DefaultAzureCredential()).create_agent(
    instructions=AGENT_INSTRUCTIONS,
    name = AGENT_NAME,
    tools=[get_random_destination]
)

In [31]:
#Option 3: Using persisted Azure AI Foundry Agents 

chat_agent : ChatAgent
credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],credential=credential)

try:
    # Create an agent that will persist
    created_agent = await project_client.agents.create_agent(
        model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS
    )
    
    chat_agent = ChatAgent(
        chat_client=AzureAIAgentClient(project_client=project_client, agent_id=created_agent.id),
        tools=[get_random_destination])
    
    print(f"Agent {created_agent.name} created, and chat agent ready")
    
finally:
    print("ok")

Agent TravelAgent created, and chat agent ready
ok


In [32]:
chat_client = OpenAIChatClient(base_url= os.environ.get("OLLAMA_ENDPOINT"),api_key="nokey", model_id=os.environ.get("OLLAMA_DEPLOYMENT_NAME"))

In [33]:
# Creating a new conversation instance with the agent storing the history of messages for this specific run
thread = chat_agent.get_new_thread()

response1 = await chat_agent.run("Plan me a day trip",thread= thread)

last_message = response1.messages[-1]
text_content = last_message.contents[0].text
print("Travel plan:")
print(text_content)

Travel plan:
Great! I’d love to help you plan a day trip. Could you please tell me which location, city, or area you’d like to visit? Or would you like me to suggest a random destination for your day trip?


In [34]:
# Adding a new user message to change the plan, on top of the past first round of conversation
# Getting the answers as a stream : 

async for chunk in chat_agent.run_stream("I don't like that destination. Plan me another vacation.", thread=thread):
    if chunk.text:
        print(chunk.text, end="", flush=True)  # 📝 Display response as it generates


[2025-11-27 17:20:51 - /workspaces/graphrag-evaluator/agent-framework-samples/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:704 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Of course! Since you haven’t specified a destination yet, would you like me to suggest a random vacation destination, or do you have a location, theme, or type of activity in mind (beaches, mountains, historical sites, etc.)? Let me know your preferences so I can tailor your vacation plan!